In [1]:
from PIL import Image
import numpy as np
import os
import torch
from diffusers import DDIMScheduler,StableDiffusionXLPipeline,StableDiffusionXLInpaintPipeline
from diffusers.utils import load_image
from torchvision.io import read_image, ImageReadMode
from masactrl.masactrl import MutualSelfAttentionControlMask_inp
from masactrl.masactrl_utils import regiter_attention_editor_diffusers
import torch.nn.functional as F
import cv2
from torchvision.utils import save_image
from torchvision.io import read_image
from pytorch_lightning import seed_everything
from matplotlib import pyplot as plt
torch.cuda.set_device(0)  # set the GPU device

/home/user01/.conda/envs/pytorch2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
image_path = "/home/user01/MyREMOVAL/examples/img/an.png"
mask_image_path = "/home/user01/MyREMOVAL/examples/mask/an_mask.png"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [6]:
scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
#scheduler = DPMSolverMultistepScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear",solver_order=3)
pipe = StableDiffusionXLInpaintPipeline.from_pretrained(
    "/home/user01/stable-diffusion-xl-base-1.0",
    #custom_pipeline="/home/user01/MyREMOVAL/pipeline_mine.py",
    scheduler=scheduler,
    variant="fp16",
    use_safetensors=True,
    torch_dtype=torch.float16,
)
pipe.to("cuda")

Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  6.75it/s]


StableDiffusionXLInpaintPipeline {
  "_class_name": "StableDiffusionXLInpaintPipeline",
  "_diffusers_version": "0.27.2",
  "_name_or_path": "/home/user01/stable-diffusion-xl-base-1.0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "requires_aesthetics_score": false,
  "scheduler": [
    "diffusers",
    "DDIMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [8]:
init_image = load_image(image_path)
mask_image = load_image(mask_image_path)
seed=775
g = torch.Generator('cuda').manual_seed(seed)

In [4]:
def load_mask(mask_path):
    mask = read_image(mask_path,mode=ImageReadMode.GRAY)
    mask = mask.unsqueeze_(0).float() / 255.  # 0 or 1
    mask = F.interpolate(mask, (512, 512))
    mask = mask.to(torch.float16)
    #mask = mask.half()
    return mask
test_mask = load_mask(mask_image_path)
# inference the synthesized image with MasaCtrl
START_STEP = 0
END_STEP = 50
LAYER = 34 #0~23down,24~33mid,34~59up
END_LAYER = 59

# hijack the attention module
editor = MutualSelfAttentionControlMask_inp(START_STEP, END_STEP, LAYER, END_LAYER, mask=test_mask,model_type="SDXL")
regiter_attention_editor_diffusers(pipe, editor)

AN at denoising steps:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
AN at U-Net layers:  [6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Using mask-guided AN


In [9]:
prompt=""
image = pipe(
    prompt=prompt,
    image=init_image,
    mask_image=mask_image,
    num_inference_steps=50,
    guidance_scale=1,
).images[0]
image.save("/home/user01/MyREMOVAL/result.png")
image

OutOfMemoryError: CUDA out of memory. Tried to allocate 512.00 MiB. GPU 0 has a total capacity of 14.58 GiB of which 465.56 MiB is free. Including non-PyTorch memory, this process has 14.12 GiB memory in use. Of the allocated memory 13.55 GiB is allocated by PyTorch, and 442.01 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)